In [1]:
pip install requests pymongo pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\pswag\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
from datetime import datetime, timezone
import requests
from pymongo import MongoClient, ASCENDING

In [4]:
APIKEY = "7bdaae7abc6622e671495ef02578a955"  # Put your API key here
MONGOURI = os.getenv("MONGOURI", "mongodb://127.0.0.1:27017/")
DBNAME = "salesweatherinfodb"
WEATHERCOL = "weather"
CITIES = ["London", "Mumbai", "New York", "Tokyo", "Sydney"]


In [7]:
def midnightutcnaive(dtutc):
    # Return midnight (00:00:00) in UTC as a naive datetime (Mongo stores as UTC)
    dtutc = dtutc.astimezone(timezone.utc)
    return datetime(dtutc.year, dtutc.month, dtutc.day, 0, 0, 0, 0)


In [9]:
def ensureindexes(col):
    # Idempotent if they already exist; MongoDB keeps them as-is.
    col.create_index([("weatherdate", ASCENDING), ("city", ASCENDING)], unique=True)
    col.create_index([("city", ASCENDING), ("weatherdate", ASCENDING)])

In [10]:
def upsertweatherdoc(doc, col):
    col.update_one(
        {"weatherdate": doc["weatherdate"], "city": doc["city"]},
        {"$set": doc},
        upsert=True
    )

In [16]:
def fetchweatherlive(apikey, city):
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": apikey.strip(), "units": "metric"}
    try:
        r = requests.get(url, params=params, timeout=20)
    except requests.RequestException as e:
        raise RuntimeError(f"Network error calling OpenWeather: {e}")

    # Fail loud with the exact API message (401, 404, 429, etc.)
    if not r.ok:
        try:
            msg = r.json()
        except Exception:
            msg = r.text
        raise RuntimeError(f"OpenWeather error {r.status_code}: {msg} url={r.url}")

    data = r.json()
    return {
        "weatherdate": datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0),
        "city": city,
        "tempc": float(data["main"]["temp"]),
        "humidity": int(data["main"]["humidity"]),
        "description": str(data["weather"][0]["description"]),
        "fetchedat": datetime.now(timezone.utc),
        "source": "openweather",
    }

In [17]:
def main():
    if not APIKEY or len(APIKEY.strip()) != 32:
        raise ValueError("OPENWEATHER API key looks invalid. Expected 32 chars.")
    client = MongoClient(MONGOURI)
    col = client[DBNAME][WEATHERCOL]
    ensureindexes(col)  # Safe even if collection/index exists

    for city in CITIES:
        try:
            doc = fetchweatherlive(APIKEY, city)
            upsertweatherdoc(doc, col)
            print(f"Stored live weather for {doc['city']} on {doc['weatherdate'].date()}.")
        except Exception as e:
            print(f"Error for {city}: {e}")

if __name__ == "__main__":
    main()

Stored live weather for London on 2025-09-21.
Stored live weather for Mumbai on 2025-09-21.
Stored live weather for New York on 2025-09-21.
Stored live weather for Tokyo on 2025-09-21.
Stored live weather for Sydney on 2025-09-21.


In [18]:
def city_daily_summary(col):
    pipeline = [
        {"$group": {
            "_id": {"city": "$city", "date": "$weatherdate"},
            "min_temp": {"$min": "$tempc"},
            "avg_temp": {"$avg": "$tempc"},
            "max_temp": {"$max": "$tempc"}
        }},
        {"$sort": {"_id.city": 1, "_id.date": 1}}
    ]
    return list(col.aggregate(pipeline))